In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import redditutils as ru
import word2vecReader as wvr

from gensim.models import Word2Vec
from gensim.models import KeyedVectors


from nltk.corpus import words
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

from skopt import BayesSearchCV
import pickle


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!ls

Model_tuning.ipynb            seinfeld-chronicles
Project Fletcher Proposal.pdf short_test_matrix.csv.mtx
README.md                     short_train_matrix.csv.mtx
Untitled.ipynb                showerthoughts-clean.ipynb
__pycache__                   showerthoughts.csv
bayes_search.pkl              test.csv
cleaned_shower.csv            test_csv.csv.
cleaning.csv                  tfidf.ipynb
darkweb-EDA.ipynb             tokenized.csv
first_5k_response.pkl         train.csv
first_5k_words.pkl            vectorized_df
fitted_cv.pkl                 vectorized_df.csv
fitted_lda_short.pkl          word2vec.ipynb
flask                         word2vecReader.py
reddit-ETL.ipynb              word2vecReaderUtils.py
redditutils.py                word2vec_twitter_model
results.csv                   word2vec_twitter_model.bin


In [7]:
df = pd.read_csv('cleaned_shower.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df['quality'] = df['score'].apply(lambda x: ru.make_labels(x))

In [38]:
df = df[df['title'].notnull()]

In [39]:
X = df['title']
y = df['quality']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 2325)

In [41]:
X_train_short = X_train[:100000]
y_train_short = y_train[:100000]

In [42]:
# English words
words_corpus = set(words.words())
# Stop words
stop = set(stopwords.words('english'))
# English words minus stop words
acceptable_words = words_corpus - stop
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

cv = CountVectorizer(stop_words='english', 
                     min_df = 2,
                     max_df = .15, 
                     tokenizer=english_corpus,
                     strip_accents='unicode',
                     encoding='utf-8', 
                     ngram_range=(1, 2))

In [44]:
X_train_short_dtm = cv.fit_transform(X_train_short)

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  sorted(inconsistent))


In [45]:
rfc = RandomForestClassifier(n_estimators = 100)

rfc_scores = cross_val_score(rfc, X_train_short_dtm, y_train_short, cv=3, scoring='roc_auc')

In [ ]:
rfc.fit(X_train_short_dtm, y_train_short)

In [ ]:
pickle.dump(rfc, open('random_forest.pkl', 'wb'))

## TFIDF Clustering

In [ ]:
# English words
words_corpus = set(words.words())
# Stop words
stop = set(stopwords.words('english'))
# English words minus stop words
acceptable_words = words_corpus - stop
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

tfidf = TfidfVectorizer(stop_words = "english",
                        strip_accents = 'ascii',
                        max_df = .10,
                        min_df = 3, 
                        tokenizer = english_corpus,
                        ngram_range=(1, 2))